In [15]:
import cudf
import dask.array as da
from cuml.linear_model import LogisticRegression
from cuml.preprocessing.model_selection import train_test_split
from sklearn.datasets import load_iris

import pandas as pd
import optuna
import numpy as np
import mlflow
import cuml
from cuml.ensemble import RandomForestClassifier
import sklearn
from cuml.metrics import accuracy_score

from multiprocessing import Manager
import random
import time

from joblib import parallel_backend
import optuna

In [16]:
from contextlib import contextmanager
import time

@contextmanager
def timed(name):
    t0 = time.time()
    yield
    t1 = time.time()
    print("..%-24s:  %8.4f" % (name, t1 - t0))

In [29]:
N_TRIALS = 10
INPUT_FILE = "/home/hyperopt/data/air_par.parquet"
n_gpu = 2

In [18]:
import time

from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

/opt/conda/envs/rapids/lib/python3.6/site-packages/distributed/node.py:155: UserWarning:

Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40011 instead



Client Scheduler: tcp://127.0.0.1:37581 Dashboard: http://127.0.0.1:40011/status,Cluster Workers: 2 Cores: 2 Memory: 49.16 GB


In [19]:
df = cudf.read_parquet(INPUT_FILE)
X, y = df.drop(["ArrDelayBinary"], axis=1), df["ArrDelayBinary"].astype('int32')

In [20]:

class Objective:
    def __init__(self, gpu_queue):
        # Shared queue to manage GPU IDs.
        self.gpu_queue = gpu_queue

    def __call__(self, trial):
        # Fetch GPU ID for this trial.
        gpu_id = self.gpu_queue.get()

        # Please write actual objective function here.
        max_depth = trial.suggest_int("max_depth", 5, 7)
        n_estimators = trial.suggest_int("n_estimators", 100, 500)

        classifier = RandomForestClassifier(max_depth=max_depth,
                             n_estimators=n_estimators)


        X_train, X_valid, y_train, y_valid = train_test_split(X, y)

        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_valid)
        score = accuracy_score(y_valid, y_pred)
        
        # Return GPU ID to the queue.
        self.gpu_queue.put(gpu_id)

        # GPU ID is stored as an objective value.
        return score

In [23]:
with timed("multi-gpu"):
    study = optuna.create_study(storage="sqlite:///example.db", direction="maximize")

    with Manager() as manager:

        # Initialize the queue by adding available GPU IDs.
        gpu_queue = manager.Queue()
        for i in range(n_gpu):
            gpu_queue.put(i)
        with parallel_backend("dask", n_jobs=n_gpu):
            study.optimize(Objective(gpu_queue), n_trials=N_TRIALS, n_jobs=n_gpu)

    # Show results.
    study.trials_dataframe()

[I 2020-06-18 18:39:23,648] A new study created with name: no-name-4862b2d8-356e-4b01-824f-4928898332f1


..multi-gpu               :   84.6736


In [24]:
params_max_depth, params_n_estimators = study.trials_dataframe()['params_max_depth'], study.trials_dataframe()['params_n_estimators']

In [30]:
with timed("sequential-call"):
    max_acc = -1
    with parallel_backend("dask", n_jobs=n_gpu):
        for i in range(N_TRIALS):
            classifier = RandomForestClassifier(max_depth=params_max_depth[i],
                                               n_estimators=params_n_estimators[i])

            X_train, X_valid, y_train, y_valid = train_test_split(X, y)

            classifier.fit(X_train, y_train)
            y_pred = classifier.predict(X_valid)
            score = accuracy_score(y_valid, y_pred)
            if score > max_acc:
                max_acc = score

..sequential-call         :   71.2088


In [31]:
print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials:  10
Best trial:
  Value:  0.8311780095100403
  Params: 
    max_depth: 7
    n_estimators: 263


In [ ]:
# def mlflow_callback(study, trial):
#     trial_value = trial.value if trial.value is not None else float("nan")
#     with mlflow.start_run(run_name=study.study_name):
#         mlflow.log_params(trial.params)
#         mlflow.log_metrics({"accuracy": trial_value})

In [ ]:
# with timed("mlflow-gpu"):
#     study = optuna.create_study(direction="maximize")
#     study.optimize(objective, n_trials=N_TRIALS, timeout=600, callbacks=[mlflow_callback])


In [ ]:
# # CPU with 750 estimators max does not finish running after hours.
# def objective_cpu(trial):
    
#     max_depth = trial.suggest_int("max_depth", 5, 15)
#     n_estimators = trial.suggest_int("n_estimators", 100, 750)

#     classifier = sklearn.ensemble.RandomForestRegressor(max_depth=max_depth,
#                                        n_estimators=n_estimators)

#     X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(X_, y_)
    
#     classifier.fit(X_train, y_train)
#     y_pred = classifier.predict(X_valid)
    
#     score = accuracy_score(y_valid, y_pred)
#     return score

In [ ]:
# with timed("cpu-etl"):
#     df_pd = pd.read_parquet(INPUT_FILE)
#     X_, y_ = df_pd.drop(["ArrDelayBinary"], axis=1), df_pd["ArrDelayBinary"].astype('int32')
    
# with timed("cpu-hpo"):
#     study = optuna.create_study(direction="maximize") # Equivalent to an experiment, a set of trials
#     study.optimize(objective_cpu, n_trials=N_TRIALS, n_jobs=-1)